In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.utils import np_utils
# from keras import optimizers
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19, preprocess_input
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.mobilenetv2 import MobileNetV2
# from keras.applications.densenet import DenseNet121
import cv2
# from keras.preprocessing.image import load_img, img_to_array


import warnings
warnings.filterwarnings('ignore')

def display_image(image, title=''):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Using TensorFlow backend.


In [2]:
#important variables
data_folder = "segmented_characters"
pickle_folder = "pickle files"
input_shape=(84, 84, 1)
extension = '.jpg'

# Display functions and others

In [3]:
def display_image(image, title='image'):
    '''
    uses openCV to display 1 image in a new window
    '''
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def display_all_in_a_list(image_list, titles=None, scale_factor=1):
    '''
    image_list: a single image, or a list of images
    -----------------------------------
    uses matplotlib to display all images in 'image_list' in the same cell
    '''
    plt.rcParams['figure.figsize'] = [10*scale_factor, len(image_list)*5*scale_factor]
    _, axes = plt.subplots(len(image_list), 1)
    if len(image_list) == 1:
        axes.imshow(image_list[0])
        if titles is not None:
            axes.set_title(titles[0])
    else:
        for index in range(len(image_list)):
            axes[index].imshow(image_list[index])
            if titles is not None:
                axes[index].set_title(titles[index])
    plt.show()

def get_rectangle_boundary(segments, min_area=100):
    '''
    returns rect_coordinates, np.array(rect_segments)
    '''
    
    new_segments = np.zeros(segments.shape, dtype=segments.dtype)
    rect_coords = []

    count = 1
    for region in regionprops(segments):
        if region.area >= min_area:
            # draw rectangle around segmented coins
            minr, minc, maxr, maxc = region.bbox
            rect_coords.append(region.bbox)
            new_segments[minr:maxr, minc:maxc] = count
            count += 1
    
    return rect_coords, new_segments

# Loading trained CNN

In [4]:
import tensorflow as tf
from keras import backend as k
 
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
 
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))


print("GPUs:", k.tensorflow_backend._get_available_gpus())

print("input shape:", input_shape)

GPUs: []
input shape: (84, 84, 1)


creating the model below

In [5]:
model_files = [file for file in os.listdir(pickle_folder)]# if file.split('_')[0]=='RandomCNN']
model_files

['group_CNNs',
 'RandomCNN_न_14.h5',
 'RandomCNN_म_16.h5',
 'RandomCNN_न_14.txt',
 'class_numerals_41.pickle',
 'RandomCNN_ऊ_30.txt',
 'vgg19',
 'RandomCNN_म_16.txt',
 'RandomCNN_ऊ_30.h5',
 'VGG19_छ_12.h5.h5',
 'VGG19_छ_12.h5']

In [6]:
%%time

import pickle
batch_size = 64

# models = []

#load models
model = load_model(os.path.join(pickle_folder, 'RandomCNN_न_14.h5'))
# model = load_model(os.path.join(pickle_folder, 'VGG19_त_19.h5'))


CPU times: user 1.66 s, sys: 416 ms, total: 2.08 s
Wall time: 2.44 s


# recognition

In [7]:
class_numerals_file = [file for file in os.listdir(pickle_folder) if file.split('_')[0]=='class']
class_numerals = pickle.load(open(os.path.join(pickle_folder, class_numerals_file[0]), 'rb'))
classes = int(class_numerals_file[0].split('_')[-1].split('.')[0])
print("classes", classes)
del class_numerals_file

def character_to_encoding(char, classes=classes):
    for i in class_numerals:
        if class_numerals[i] == char:
            return np_utils.to_categorical(i, classes)

def encoding_to_character(encoding):
    return class_numerals[encoding.argmax()]

classes 41


for i in range(4):
    image = cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(data_folder, str(i)+extension)), cv2.COLOR_RGB2GRAY), (input_shape[0], input_shape[1]))
    display_image(image)

In [9]:
character_files = os.listdir(data_folder)
max_char = max(list(map(lambda x: int(x.split('.')[0]), os.listdir(data_folder))))

for i in range(max_char):
    if str(i) + extension in character_files:
        x = cv2.imread(os.path.join(data_folder, str(i)+extension))        
        char_image = cv2.resize(cv2.cvtColor(x, cv2.COLOR_RGB2GRAY), (input_shape[0], input_shape[1]))
#         char_image = cv2.resize(preprocess_input(x), (input_shape[0], input_shape[1]))
#         display_image(char_image)
#         break
        prediction = model.predict((char_image/255).reshape((1, input_shape[0], input_shape[1], 1)))
        prediction[prediction>0.5] = 1
        prediction[prediction<0.5] = 0
        print(encoding_to_character(prediction), end=' ')
    else:
        print()
#     break
# predictions = [models[i].predict(X_test, batch_size=batch_size) for i in range(3)]

प स च च 
च च च म 
म 
च च 
च प ए प 
प ए 
क म 
च ह 
थ 
प च 
च 
ष च 
प र 
ब र म 
अ 
च ट 
ड ए 
प न 
म च च च 
म 
च ज 
थ 
ब च च क 
प स 
क 
म च ब च 
प च 
क ट च ब प 
म स ज 
क 
र व 
ए च 
च च 
च 
क 
फ छ च 
क 
ङ प च 
च च च 
प ए 
स च 
च 
प स 
च प ट 
क च 
म च इ च 
क 
प स 
प ड च 
ब र 
च क 
क य 
स 
च अ 
प ए 
प च 
क 
म स च 
ब घ 
